## Monte Carlo simulation

## 1. environment setup

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style('whitegrid')

In [2]:
# project variables/historical variables
# project budgeted vs. actual has an average of 100% achievement, which is set to 1

avg = 1
std_dev = 0.13
num_projects = 1024
num_simulations = 1000 

## 2. generate data

In [3]:
# randomly generate a list of project achievement probabilities % for all projects, based off historical data

pct_to_target = np.random.normal(avg, std_dev, num_projects).round(2)

In [4]:
pct_to_target 

array([0.95, 0.95, 1.01, ..., 1.1 , 0.73, 1.24])

In [5]:
# randomly generate a list of project targets
# for example, there is a 40% chance of getting a $750,000 project
# for demonstration purpose only, this can be fine-grained

project_target_values = [500_000, 750_000, 1_000_000, 1_500_000, 2_000_000]
project_target_prob = [.3, .4, .2, .1, .1]
project_target = np.random.choice(project_target_values, num_projects, project_target_prob)

In [6]:
# setup data table
df = pd.DataFrame(index=range(num_projects), data = {'Achievement': pct_to_target, 'Project Budgeted': project_target})
df['Project Actual'] = df['Achievement'] * df['Project Budgeted']

In [7]:
# NOTE: statistically, although project actual may vary significantly, the achievement probability remains consistent

## 3. financial model

In [8]:
# the financial model should be designed for specific use
# objective: calculate gross profit
# assumptions: 1) projects that miss budget and 2 )has been budgeted above 100,000,000 
#              will incurr 77% COGS due to overhead/back office expenses; otherwise, 72% COGS

def calculate_cogs(achievement, budget):
    if (achievement < 1.0) & (budget > 1_000_000):
        return 0.77
    else:      
        return 0.72

df['COGS_Rate'] = df[['Achievement','Project Budgeted']].apply(lambda x: calculate_cogs(x['Achievement'], x['Project Budgeted']), axis=1)
df['GP_Amount'] = df['Project Actual'] * (1 - df['COGS_Rate'])

## 4. Monte Carlo simulation

In [9]:
result = []

for i in range(num_simulations):
    # randomly generate based on historical data
    project_target = np.random.choice(project_target_values, num_projects, project_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_projects).round(2)
    
    # calculate project actual achievement amount
    df = pd.DataFrame(index=range(num_projects), data = {'Achievement': pct_to_target, 'Project Budgeted': project_target})
    df['Project Actual'] = df['Achievement'] * df['Project Budgeted']
    
    # as above, determine the GP
    df['COGS_Rate'] = df[['Achievement','Project Budgeted']].apply(lambda x: calculate_cogs(x['Achievement'], x['Project Budgeted']), axis=1)
    df['GP_Amount'] = df['Project Actual'] * (1 - df['COGS_Rate'])
    
    # store into result set
    result.append([df['Project Actual'].sum().round(0),
                   df['GP_Amount'].sum().round(0),
                   df['Project Budgeted'].sum().round(0)])

In [10]:
result = pd.DataFrame.from_records(result, columns=['Project Actual', 'GP_Amount', 'Project Budgeted'])

## 5. Getting Results

In [26]:
result.describe().style.format('{:,}').highlight_max(axis=1)

,Project Actual,GP_Amount,Project Budgeted
count,"1,000.0","1,000.0","1,000.0"
mean,"1,177,832,905.0","314,238,714.4","1,177,874,750.0"
std,"18,407,428.871888954","4,873,800.885252913","17,525,282.036188304"
min,"1,126,387,500.0","300,958,000.0","1,126,000,000.0"
25%,"1,164,796,875.0","310,826,562.5","1,166,000,000.0"
50%,"1,178,066,250.0","314,400,525.0","1,177,750,000.0"
75%,"1,189,838,125.0","317,459,500.0","1,189,250,000.0"
max,"1,225,997,500.0","327,715,350.0","1,233,500,000.0"
